# Prétraitement des données

## Suppression des éléments indésirables de nos tweets

In [1]:
# Imports nécessaires
import pandas as pd
import numpy as np
import string 
import re
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

In [38]:
# Lecture et stockage de la base de données
tweet_df = pd.read_pickle('../../delphes/data/final4_clean.csv')
tweet_df.head()

,mep_id,name,country,group,nat_group,twitter,content
0,197490,Magdalena ADAMOWICZ,Poland,Group of the European People's Party (Christia...,Independent,Adamowicz_Magda,[@danutahuebner Bardzo dziękuję @danutahuebner...
1,189525,Asim ADEMOV,Bulgaria,Group of the European People's Party (Christia...,Citizens for European Development of Bulgaria,AdemovAsim,[135 години единна и силна България. \nЧестит ...
2,124831,Isabella ADINOLFI,Italy,Non-attached Members,Movimento 5 Stelle,Isa_Adinolfi,"[Sembra un film, ma purtroppo è realtà: le imm..."
6,125045,Clara AGUILERA,Spain,Group of the Progressive Alliance of Socialist...,Partido Socialista Obrero Español,ClaraAguilera7,"[Clara Aguilera: ""El criterio científico debe ..."
7,204335,Alviina ALAMETSÄ,Finland,Group of the Greens/European Free Alliance,Vihreä liitto,alviinaalametsa,[@LHurttila @MariaOhisalo Kaupungilla on järje...


In [ ]:
age = []
sexe = []

In [85]:
y = np.arange(0, tweet_df.shape[0])
tweet_df['index'] = y
tweet_df.set_index('index', inplace=True)

In [87]:
new_test_df = pd.DataFrame(columns=['mep_id', 'name', 'country', 'group', 'nat_group', 'twitter', 'content'])

In [130]:
%%time
k = 0
for j in range(tweet_df.shape[0]):
    for i in range(len(tweet_df['content'][j])):
        new_test_df.loc[k] = tweet_df.loc[j,'mep_id':'twitter']
        new_test_df.loc[k, 'content'] = tweet_df['content'][j][i]
        k += 1

Wall time: 54min 54s


In [120]:
# Remove the undesirable elements in the entire dataframe
def rmurl_df(df, column_name):
    '''
    This function removes all the URLs, the #hashtag and the @user of a column made of strings.
    Be careful to apply it BEFORE all the other preprocessing steps (if not it wont'
    be recognized as a URL)
    '''
    df = df.copy()
    df[column_name] = df[column_name].str.replace('http\S+|www.\S+|@\S+|#\S+', '', case=False)
    return df



In [191]:
new_test_df.to_pickle('~')

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\psmag'

In [121]:
# Lowercase the tweet's column
def lower_df(df, column_name):
    '''
    This function lowercases a column made of strings.
    '''
    df = df.copy()
    df[column_name] = df[column_name].str.lower()
    return df

In [122]:
# Remove the numbers in the tweet's column
def rmnumbers_df(df, column_name):
    '''
    This function removes all the digits of a column made of strings.
    '''
    df = df.copy()
    def remove_numbers(text):
        return ''.join(word for word in text if not word.isdigit())
    df[column_name] = df[column_name].apply(remove_numbers)
    return df

In [123]:
# Remove the undesirable punctuations in the tweet's column
def rmpunct_df(df, column_name):
    '''
    This function removes all the punctuations, all the "rt" and remove multiple spaces
    of a column made of strings.
    '''
    punct = string.punctuation
    df = df.copy()
    def replace_punct(text):
        for punctu in punct:
            text = text.replace(punctu, ' ')
        text = text.replace(' rt ','')
        text = " ".join(text.split())
        return text
    df[column_name] = df[column_name].apply(replace_punct)
    return df

In [124]:
# Remove the stopwords in the tweet's column
def rmstopwords_df(df, column_name):
    '''
    This function removes all the stopwords of a column made of strings.
    '''
    df = df.copy()
    stop_words = stopwords.words('english')
    def remove_stopwords(text):
        for word in stop_words:
            text = text.replace(f' {word} ', ' ')
        return text
    df[column_name] = df[column_name].apply(remove_stopwords)
    return df

In [125]:
# Remove the undesirable emojis in the entire dataframe
def rmemojis_df(df):
    '''
    This function removes all the emojis of a column made of strings.
    Be careful to translate in latin alphabet before applying this function : 
    it also removes cyrillic alphabet
    '''
    df = df.copy()
    df = df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
    return df

In [134]:
clean_df = rmurl_df(new_test_df, 'content')
clean_df = lower_df(clean_df, 'content')
clean_df = rmnumbers_df(clean_df, 'content')
clean_df = rmpunct_df(clean_df, 'content')
clean_df = rmstopwords_df(clean_df, 'content')
clean_df = rmemojis_df(clean_df)

In [138]:
clean_df[clean_df['country'] == 'Ireland'].head()

,mep_id,name,country,group,nat_group,twitter,content
24722,124988,Deirdre CLUNE,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,deirdreclunemep,the northern ireland protocol must protected l...
24723,124988,Deirdre CLUNE,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,deirdreclunemep,as member special committee beating cancer loo...
24724,124988,Deirdre CLUNE,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,deirdreclunemep,this month shining light childrens cancer parl...
24725,124988,Deirdre CLUNE,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,deirdreclunemep,yesterday told europe need keep speed amp infr...
24726,124988,Deirdre CLUNE,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,deirdreclunemep,this letter sent taoiseach clear seeking clari...


In [139]:
clean_df['content']

0         bardzo dzikuj niezalene od wadzy wolne media d...
1         komisja przyja arcywane projekty pilotaowe aut...
2         komisja przyja projekty pilotaowe mojego wspau...
3         w tym dniu w tym miejscu w tej godzinie prosz ...
4         bg nie potrzebuje by przez nikogo broniony nie...
                                ...                        
137295    tedenska akcija od etrtka srede zbornik bela k...
137296    strong amp way forward european future read eu...
137297    a date noted worrying state danger ends hu spe...
137298    ko vsi vedo da je nekaj noro pa se iz strahu p...
137299    tota ekipa tak se gre iz kranja z najlepim nav...
Name: content, Length: 137300, dtype: object

In [160]:
testbase_df = clean_df[clean_df['country'] == 'Ireland']

## Premier test Word2Vec

In [161]:
testbase_df = testbase_df[['name', 'content']]

In [162]:
from gensim.models import Word2Vec

In [177]:
sentences = testbase_df['content']

In [178]:
y_train = pd.get_dummies(testbase_df['name'])

In [179]:
sentences_inter = []
for sentence in sentences:
    sentences_inter.append(sentence.split())

In [181]:
from sklearn.model_selection import train_test_split
sentences_train, sentences_test = train_test_split(sentences_inter, batch_size = 0.3)

TypeError: Invalid parameters passed: {'batch_size': 0.3}

In [150]:
word2vec = Word2Vec(sentences=sentences_train)

In [92]:
def embed_sentence(word2vec, sentence):
    y = []
    for word in sentence:
        if word in word2vec.wv.vocab.keys():
           y.append(word2vec[word])
    return np.array(y)

def embedding(word2vec, sentences):
    
    y = []
    for sentence in sentences:
        y.append(embed_sentence(word2vec, sentence))
    return y

In [151]:
X_train = embedding(word2vec,sentences_train)
X_test = embedding(word2vec,sentences_test)

c:\users\psmag\.venvs\delphes\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [158]:
from tensorflow.keras.utils import to_categorical
y_train = pd.get_dummies(testbase_df['mep_id'])

In [95]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

def init_model():

    model = Sequential()
    model.add(layers.Masking(mask_value = -1000))
    model.add(layers.LSTM(13))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop', 
              metrics=['accuracy'])
    return model

model = init_model()

In [96]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train_pad = pad_sequences(X_train, padding='post',value=-1000, dtype='float32')

In [97]:
X_train_pad.shape

(9, 4608, 100)

In [98]:
y_cat_train.shape

(9, 9)

In [99]:
model = init_model()
model.fit(X_train_pad, y_cat_train, epochs=100)

Train on 9 samples
Epoch 1/100
9/9 [==============================] - 3s 353ms/sample - loss: 2.1945 - acc: 0.1111
Epoch 2/100
9/9 [==============================] - 3s 287ms/sample - loss: 2.1867 - acc: 0.1111
Epoch 3/100
9/9 [==============================] - 2s 272ms/sample - loss: 2.1793 - acc: 0.1111
Epoch 4/100
9/9 [==============================] - 2s 276ms/sample - loss: 2.1701 - acc: 0.1111
Epoch 5/100
9/9 [==============================] - 2s 276ms/sample - loss: 2.1626 - acc: 0.1111
Epoch 6/100
9/9 [==============================] - 2s 274ms/sample - loss: 2.1555 - acc: 0.1111
Epoch 7/100
9/9 [==============================] - 2s 274ms/sample - loss: 2.1495 - acc: 0.1111
Epoch 8/100
9/9 [==============================] - 2s 273ms/sample - loss: 2.1418 - acc: 0.1111
Epoch 9/100
9/9 [==============================] - 2s 271ms/sample - loss: 2.1344 - acc: 0.1111
Epoch 10/100
9/9 [==============================] - 2s 273ms/sample - loss: 2.1271 - acc: 0.1111
Epoch 11/100
9/9 [==

Epoch 86/100
9/9 [==============================] - 2s 271ms/sample - loss: 1.7257 - acc: 0.3333
Epoch 87/100
9/9 [==============================] - 2s 273ms/sample - loss: 1.7178 - acc: 0.4444
Epoch 88/100
9/9 [==============================] - 2s 268ms/sample - loss: 1.7118 - acc: 0.3333
Epoch 89/100
9/9 [==============================] - 2s 266ms/sample - loss: 1.7039 - acc: 0.4444
Epoch 90/100
9/9 [==============================] - 2s 275ms/sample - loss: 1.6970 - acc: 0.3333
Epoch 91/100
9/9 [==============================] - 3s 292ms/sample - loss: 1.6897 - acc: 0.3333
Epoch 92/100
9/9 [==============================] - 3s 284ms/sample - loss: 1.6832 - acc: 0.3333
Epoch 93/100
9/9 [==============================] - 3s 291ms/sample - loss: 1.7013 - acc: 0.3333
Epoch 94/100
9/9 [==============================] - 2s 270ms/sample - loss: 1.7372 - acc: 0.2222
Epoch 95/100
9/9 [==============================] - 2s 272ms/sample - loss: 1.6858 - acc: 0.4444
Epoch 96/100
9/9 [============

In [107]:
sentences[669]

'want know alternatives college talk options chat us online epic wow thank including leading ladies  visit must trust british government implement withdrawal agreement obligation international law amp prerequisite future partnership protocol ireland northern ireland essential protect peace stability island amp integrity single market absolutely sheila absolutely  years class could summed word would resilience n ncancelled exams home months online learning pandemic youve year go decade n nheres next adventure  ud wishing students best luck tomorrow n nresults available tomorrow congratulations constituency colleague nominated represent ireland commissioner level dedicated eu respective roles within next steps candidates speak  congratulations lady amazing news  great post  well done paving way stronger relationship eu neighbours grateful opportunity present pre budget submission soon policies youth minimum wage equality bystander intervention programmes universities amp secondary school

In [108]:
examples2 = ['morning Deputy Dara Calleary tendered resignation Minister Agriculture Food Marine following attendance Oireachtas golf dinner Wednesday evening attendance event wrong error judgement part accepted resignation People country have made difficult personal sacrifices family lives businesses comply COVID-19 regulations event should gone ahead manner given Government decision last Tuesday Dara Calleary since first elected Dail Eireann remains committed dedicated public representative error judgement character made right decision country particularly light continued efforts suppress COVID-19',
            sentences[669]]

In [109]:
examples2[0] = examples2[0].lower()

In [110]:
examples=[]
for example in examples2:
    examples.append(example.split())

In [111]:
examples[1]

['want',
 'know',
 'alternatives',
 'college',
 'talk',
 'options',
 'chat',
 'us',
 'online',
 'epic',
 'wow',
 'thank',
 'including',
 'leading',
 'ladies',
 'visit',
 'must',
 'trust',
 'british',
 'government',
 'implement',
 'withdrawal',
 'agreement',
 'obligation',
 'international',
 'law',
 'amp',
 'prerequisite',
 'future',
 'partnership',
 'protocol',
 'ireland',
 'northern',
 'ireland',
 'essential',
 'protect',
 'peace',
 'stability',
 'island',
 'amp',
 'integrity',
 'single',
 'market',
 'absolutely',
 'sheila',
 'absolutely',
 'years',
 'class',
 'could',
 'summed',
 'word',
 'would',
 'resilience',
 'n',
 'ncancelled',
 'exams',
 'home',
 'months',
 'online',
 'learning',
 'pandemic',
 'youve',
 'year',
 'go',
 'decade',
 'n',
 'nheres',
 'next',
 'adventure',
 'ud',
 'wishing',
 'students',
 'best',
 'luck',
 'tomorrow',
 'n',
 'nresults',
 'available',
 'tomorrow',
 'congratulations',
 'constituency',
 'colleague',
 'nominated',
 'represent',
 'ireland',
 'commissione

In [112]:
X_test = embedding(word2vec, examples)
X_test_pad = pad_sequences(X_test, padding='post',value=-1000, dtype='float32')

c:\users\psmag\.venvs\delphes\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [113]:
model.predict(X_test_pad)

array([[0.05189741, 0.1010161 , 0.09059594, 0.16365586, 0.15643859,
        0.15870222, 0.08425017, 0.11839338, 0.07505037],
       [0.05135225, 0.09798919, 0.11039513, 0.17092562, 0.14136548,
        0.14295104, 0.10722873, 0.10067838, 0.07711402]], dtype=float32)

In [117]:
sentences

136    the northern ireland protocol must protected l...
146    mornings like hard brexit looms youd miss calm...
208    important timely message  n n words justice ye...
209    convergence agri committee maybe like last twe...
335    if uk gov tries contravene brings serious ques...
404    happening morning edt join us paula dobriansky...
405    i raising mooted levy called fast fashion eu c...
478    starting today join discussions issues around ...
668    vote ep environment public health committee ev...
669    want know alternatives college talk options ch...
Name: content, dtype: object

In [119]:
clean_df.loc[209]

mep_id                                                  124985
name                                        Luke Ming FLANAGAN
country                                                Ireland
group        Group of the European United Left - Nordic Gre...
nat_group                                          Independent
twitter                                               lukeming
content      convergence agri committee maybe like last twe...
Name: 209, dtype: object